In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from utils import *
warnings.filterwarnings('ignore')

In [ ]:
logger = Logger(log_level="DEBUG")

INPUT_DATA_DIR = 'output'
OUTPUT_DATA_DIR = 'output'

full_data = pd.read_csv(f'{OUTPUT_DATA_DIR}/full_data.csv', index_col=False)
full_data['Date'] = pd.to_datetime(full_data['Date'])

In [ ]:
full_data.head(2)

In [ ]:
filter_info = {
    'acc_holder': 'any', # ['sachin', 'nowrin', 'joint', 'any']
    'date_period': {
        'type': 'month', # ['all', 'from-to', 'month']
        'year': 2023,
        'month': 10
    },
}

data = filter_data_by_info(full_data, filter_info)
data.head(2)

In [ ]:
category_info = pd.read_csv(f'utils/category.csv')
category_info.head(5)

In [ ]:
data[['Category', 'Subcategory']] = data.apply(
    find_category, axis=1, result_type='reduce', args=(category_info,))
data.head(2)

In [ ]:
unassigned_data = data.query("Category == 'others'")
logger.info(f"Unass. data count : {100*len(unassigned_data)/len(data) :.2f} %")
logger.info(f"Unass. data trans.: {100*unassigned_data['Amount'].abs().sum()/data['Amount'].abs().sum() :.2f} %")

In [ ]:
if filter_info['date_period']['type'] == 'month':
    YEAR = filter_info['date_period']['year']
    MONTH = filter_info['date_period']['month']
    # data.to_csv(f'{OUTPUT_DATA_DIR}/{YEAR}_{MONTH}_notes.csv', index=False)
    write_to_excel(data,  f"{OUTPUT_DATA_DIR}/{YEAR}_{MONTH}_notes.xlsx")
elif filter_info['date_period']['type'] == 'all':
    write_to_excel(data,  f"{OUTPUT_DATA_DIR}/full_data_notes.xlsx")
else:
    pass

In [ ]:
def describe_pie(df):
    display(df.to_frame().T)
    plot_pie(df)

In [ ]:
all_expenses = data.pivot_table(values='Amount', index='Category', aggfunc='sum').squeeze().sort_values(ascending=False)
relevant_expenses = all_expenses.drop(index=['salary', 'others', 'transfer'], errors='ignore') # , 'house'
describe_pie(relevant_expenses)

In [ ]:
groceries_expenses = data.query("Category == 'groceries'").pivot_table(values='Amount', index='Subcategory', aggfunc='sum').squeeze().sort_values(ascending=False)
describe_pie(groceries_expenses)

In [ ]:
restaurant_expenses = data.query("Category == 'restaurant'").pivot_table(values='Amount', index='Subcategory', aggfunc='sum').squeeze().sort_values(ascending=False)
describe_pie(restaurant_expenses)

In [ ]:
filter_info = {
    'acc_holder': 'any', # ['sachin', 'nowrin', 'joint', 'any']
    'date_period': {
        'type': 'all', # ['all', 'from-to', 'month']
    },
}

data = filter_data_by_info(full_data, filter_info)
data[['Category', 'Subcategory']] = data.apply(
    find_category, axis=1, result_type='reduce', args=(category_info,))

In [ ]:
relevant_expenses

In [ ]:
all_expenses = data.pivot_table(values='Amount', index='Category', aggfunc='sum').squeeze().sort_values(ascending=False)
relevant_expenses = all_expenses.drop(index=['salary', 'others', 'transfer'], errors='ignore') # , 'house'
describe_pie(relevant_expenses)

In [ ]:
write_to_excel(data,  f"{OUTPUT_DATA_DIR}/full_data_notes.xlsx")

In [ ]:
def visualize_gas(data):
    interested_data = data.query("Category == 'gas'")
    interested_data['Amount'] = -interested_data['Amount']
    interested_data['Date'] = pd.to_datetime(interested_data['Date'])
    interested_data['YearMonth'] = interested_data['Date'].dt.to_period('M')
    monthly_sum = interested_data.groupby('YearMonth')['Amount'].sum().reset_index()
    monthly_sum['YearMonth'] = monthly_sum['YearMonth'].dt.to_timestamp()
    plt.figure(figsize=(10, 6))
    plt.plot(monthly_sum['YearMonth'], monthly_sum['Amount'], marker='o', linestyle='-')
    plt.title('Monthly Amounts')
    plt.xlabel('Month')
    plt.ylabel('Total Amount')
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

visualize_gas(data)

Other functions:

* `visualize_salary(data)`: to see salary trajectory across months
* `visualize_salary(food)`: to see groceries and restaurant trajectory across months